In [ ]:
# default_exp timeseries.data

# timeseries.data

> API details.

In [ ]:
#export
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.tabular.data import *
from fastai.tabular.core import *
from fastrenewables.tabular.core import *
from fastrenewables.timeseries.core import *
import glob

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
class RenewableTimeSeriesDataLoaders(DataLoaders):
    """Creates a timerseries data and dataloaders."""
    @classmethod
    @delegates(Tabular.dataloaders, but=["dl_type", "dl_kwargs"])
    def from_df(cls, df, path='.', procs=None, pre_procs=None, \
                cat_names=None, cont_names=None, \
                y_names=None, y_block=RegressionBlock(),\
                splits=None, **kwargs):
        
        "Create from `df` in `path` using `procs`"
        if cat_names is None: 
            cat_names = []
            
        if cont_names is None: 
            cont_names = list(set(df)-set(L(cat_names))-set(L(y_names)))
        
        if pre_procs is None: 
            pre_procs = [CreateTimeStampIndex("TimeUTC"), AddSeasonalFeatures]
            
        if procs is None:
            procs = [NormalizePerTask, Categorify]
            
        to = TabularRenewables(
            df,
            cont_names=cont_names,
            cat_names=cat_names,
            y_names=y_names,
            pre_process=pre_procs,
            procs=procs,
            y_block=y_block,
            splits=None,
        )
        splits = RandomSplitter(valid_pct=0.2) if splits is None else splits
        
        tt = Timeseries(to, splits=splits, **kwargs)
        
        return TimeSeriesDataLoaders(tt, **kwargs)
    
    @classmethod
    def from_files(cls, files,  **kwargs):
        dfs = read_files(files)
        
        dfs = pd.concat(dfs, axis=0)
#         if "cat_names" in kwargs.keys():
#             kwargs["cat_names"] = kwargs["cat_names"] if "TaskID" in kwargs["cat_names"] else kwargs["cat_names"] + ["TaskID"]
#         else:
#             kwargs["cat_names"] = ["TaskID"]
        
        return cls.from_df(dfs, **kwargs)

In [ ]:
files = glob.glob("../data/*.h5"); len(files)

3

In [ ]:
cont_names = ['T_HAG_2_M', 'RELHUM_HAG_2_M', 'PS_SFC_0_M', 'ASWDIFDS_SFC_0_M',
       'ASWDIRS_SFC_0_M', 'WindSpeed58m',
       'SinWindDirection58m', 'CosWindDirection58m', 'WindSpeed60m',
       'SinWindDirection60m', 'CosWindDirection60m', 'WindSpeed58mMinus_t_1',
       'SinWindDirection58mMinus_t_1', 'CosWindDirection58mMinus_t_1',
       'WindSpeed60mMinus_t_1', 'SinWindDirection60mMinus_t_1',
       'CosWindDirection60mMinus_t_1', 'WindSpeed58mPlus_t_1',
       'SinWindDirection58mPlus_t_1', 'CosWindDirection58mPlus_t_1',
       'WindSpeed60mPlus_t_1', 'SinWindDirection60mPlus_t_1',
       'CosWindDirection60mPlus_t_1']

cat_names = ['TaskID', 'Month', 'Day', 'Hour']

In [ ]:
pd.options.mode.chained_assignment=None
dls = RenewableTimeSeriesDataLoaders.from_files(glob.glob("../data/*.h5"), y_names="PowerGeneration", 
                                      pre_procs=[FilterYear(year=2020), 
                                                 AddSeasonalFeatures(as_cont=False),
                                                FilterInconsistentSamplesPerDay], 
                                      cat_names=cat_names, cont_names=cont_names, bs=13)

In [ ]:
cats, conts, ys = dls.one_batch()

In [ ]:
cats.shape, conts.shape, ys.shape

(torch.Size([13, 4, 24]), torch.Size([13, 23, 24]), torch.Size([13, 1, 24]))